In [ ]:
from lstm_model import build_open_loop_lstm, load_open_loop_lstm
from data_processing import create_training_split, df_training_split, train_valid_test_split, df_train_valid_test_split, create_df_3d, create_window_closed_loop, add_new_pred, compute_lyapunov_time_arr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime, time
import tensorflow_datasets as tfds
import tensorflow as tf
import lstm_model
import importlib
import Feedback_Model
import data_processing
import plots
plt.rcParams["figure.facecolor"] = "w"
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
importlib.reload(Feedback_Model)

In [ ]:
mydf = np.genfromtxt('CSV/Lorenz_trans_001.csv', delimiter=',')
time = mydf[0, :]
mydf = mydf[1:, :]
print("Shape of discrete time step array ", time.shape)
print("Shape of solution array: ", mydf.shape)
df_train, df_valid, df_test = df_train_valid_test_split(mydf)
time_train, time_valid, time_test = train_valid_test_split(time)
x_train, x_valid, x_test = train_valid_test_split(mydf[0, :])
y_train, y_valid, y_test = train_valid_test_split(mydf[1, :])
z_train, z_valid, z_test = train_valid_test_split(mydf[2, :])
window_size = 50
batch_size = 32
shuffle_buffer_size = df_train.shape[0]
train_dataset = create_df_3d(df_train.transpose(), window_size, batch_size, shuffle_buffer_size)
valid_dataset = create_df_3d(df_valid.transpose(), window_size, batch_size, 1)
test_dataset = create_df_3d(df_test.transpose(), window_size, batch_size, 1)

In [ ]:
# only take first element of dataset
for data, labels in train_dataset.take(1):
    numpy_data = data.numpy()
    numpy_labels = labels.numpy()
    

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss', patience=5)
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)
history = model_oloop.fit(train_dataset, epochs=100, batch_size=batch_size,
                          validation_data=valid_dataset, callbacks=[tensorboard_callback])  # , early_stop_callback])

In [ ]:
def custom_loss(y_true, y_pred, washout=0):
    mse = tf.keras.losses.MeanSquaredError()
    # (batchsize, dimensions)
    loss = mse(y_true[washout:, :], y_pred[washout:, :])
    return loss

In [ ]:
feedback_model = Feedback_Model.FeedBack(units=10)

In [ ]:
feedback_model.fit(train_dataset, epochs=10, batch_size=32,
                   validation_data=valid_dataset, verbose=2)

In [ ]:
pred = plots.plot_closed_loop_lya(
    feedback_model,
    2,
    time_test,
    df_test,
    img_filepath=None,
    n_length=500,
    window_size=50,
)

In [ ]:
n_epochs = 0 + 100
n_batches = 180
cloop_size = 32 * n_batches
test_window, labels, idx = lstm_model.select_random_batches_with_label(
    df_train.transpose(), n_batches
)
predictions = feedback_model.predict(np.array(test_window).reshape(cloop_size, 50, 3))
cloop_windows, cloop_label = lstm_model.add_cloop_prediction(
    df_train.transpose(), idx, np.array(predictions).reshape((cloop_size, 1, 3))
)
cloop_dataset = tf.data.Dataset.from_tensor_slices((cloop_windows, cloop_label.reshape(cloop_size,3)))

In [ ]:
cloop_dataset = cloop_dataset.padded_batch(32, padded_shapes=([None, 3], [None]))

In [ ]:
feedback_model.train_cloop(train_dataset, cloop_dataset, 5, 2, valid_dataset)

In [ ]:
pred = plots.plot_closed_loop_lya(
    feedback_model,
    50,
    time_test,
    df_test,
    img_filepath=None,
    n_length=500,
    window_size=50,
)